# High energy neutrino interactions in a massive target

We want to compute how many neutrinos would interact, knowing the fluxes of incoming neutrinos
## Importing the data

We use here spectra of neutrino incoming in the detector, taken from the SHiP experiment. Data are in the CSV format (Comma Separated Values), where the values are separated by a comma:

x,y

x1,y1

...


In [1]:
TGraph *nuflux = new TGraph("nu_mu.csv","%lg %lg","\t,");

First, let us draw the flux:

In [2]:
%jsroot on
//inserting some labels
nuflux->SetTitle("Input Neutrino spectrum");
nuflux->GetXaxis()->SetTitle("E[GeV]");
nuflux->GetYaxis()->SetTitle("counts");

nuflux->SetEditable(kFALSE);

In [3]:
//setting marker style and size
gStyle->SetMarkerStyle(2);
gStyle->SetMarkerSize(1);

In [4]:
//launching drawing commands
TCanvas *c = new TCanvas();
nuflux->Draw("AP*");
c->Draw();

## Setting Target Parameters
The SHiP neutrino target is made of *bricks*, interleaving nuclear emulsion with passive material.

Let us assume the passive material is **lead**:

In [5]:
const int A = 208; //initial approximation, only pb208 considered

Then we need mass and transverse size of the target

In [6]:
const double mass = 8183*1e+3; //mass in grams
const double surface = 90.3 * 74.9; //surface in square centimetres

## Computing the cross section
We will focus here on high energy interaction:

In [7]:
const double Emin = 50.;

Here the cross section dependance on energy is approximatively linear (CC-DIS is dominant):

$$
\sigma^{CC}_\nu (E)= \sigma^{const}_\nu * E 
$$

This is the cross section per nucleon.
We need to do:

$$
N_{int}(E) = \frac{N}{S} * \bigg(\frac{m}{A} * N_A\bigg) * \bigg(\sigma^{cc}_\nu(E) * A\bigg)
$$

In [8]:
const double avogadro = 6.022e+23;
const double sigmaoverE = 0.68e-38; // cm^2/GeV

In [9]:
double Nint = 0.;
double Ntotint = 0.;

We will do this product bin by bin, then sum all the results. This will get us the total number of interacting neutrinos:

In [10]:
TGraph *nuinteracting = new TGraph();
int pointcounter = 0; //TGraph points need to be numbered
for(int ibin = Emin; ibin<400;ibin++){
    double x = ibin + 0.5; //points are at half energy bins
    //getting flux
    double F = (nuflux->Eval(x)/surface);
    
    //getting cross section
    double sigma = sigmaoverE *x;
    
    Nint = F * mass/A * avogadro *A * sigma; 
    Ntotint += Nint;
    
    nuinteracting->SetPoint(pointcounter,x,Nint);
    pointcounter += 1;
}

In [11]:
TCanvas *cinteracting = new TCanvas();
nuinteracting->SetTitle("Neutrino interacting spectrum");
nuinteracting->GetXaxis()->SetTitle("E[GeV]");
nuinteracting->GetYaxis()->SetTitle("counts");

//setting marker style and size
TCanvas *c2 = new TCanvas();
nuinteracting->Draw("AP*");
nuinteracting->SetEditable(kFALSE);
c2->Draw();

Finally, let us display how many neutrinos interacted in total:

In [12]:
cout<<TString::Format("Interacting in total: %.2e",Ntotint).Data()<<endl;

Interacting in total: 8.52e+05
